In [ ]:
! pip install --upgrade pip 

In [ ]:
! pip uninstall feast --yes
! pip install feast --upgrade --no-cache-dir
! pip install "feast[redis,postgres,snowflake]" --upgrade --no-cache-dir

In [ ]:

! pip install cryptography==38.0.4 --no-cache-dir
! pip install pyOpenSSL==22.1.0 --no-cache-dir 
! pip install proto-plus==1.22.0 --no-cache-dir 
! pip install protobuf==3.20.1 --no-cache-dir 

In [ ]:

! pip uninstall typing_extensions --y
! pip install typing_extensions==4.4.0

import importlib
import typing_extensions

importlib.reload(typing_extensions)


In [ ]:
! pip uninstall feast --yes
! pip install ./feast/feast --upgrade --no-cache-dir

In [ ]:
!feast version

In [ ]:
import boto3
ssm = boto3.client('ssm')
postgresdb = ssm.get_parameter(Name='/data_platform/dev/featurestore/postgresdatabsename', WithDecryption=True)['Parameter']['Value']
postgrespwd = ssm.get_parameter(Name='/data_platform/dev/featurestore/postgrespassword', WithDecryption=True)['Parameter']['Value']
redisconn = ssm.get_parameter(Name='/data_platform/dev/featurestore/redisconnection', WithDecryption=True)['Parameter']['Value']
redispwd = ssm.get_parameter(Name='/data_platform/dev/featurestore/redispassword', WithDecryption=True)['Parameter']['Value']
snowflakeaccount = ssm.get_parameter(Name='/data_platform/dev/featurestore/snowflakeaccount', WithDecryption=True)['Parameter']['Value']
snowflakepwd = ssm.get_parameter(Name='/data_platform/dev/featurestore/snowflakepassword', WithDecryption=True)['Parameter']['Value']
snowflakeuser = ssm.get_parameter(Name='/data_platform/dev/featurestore/snowflakeuser', WithDecryption=True)['Parameter']['Value']
snowflakedb = ssm.get_parameter(Name='/data_platform/dev/featurestore/snowflakewaredatabse', WithDecryption=True)['Parameter']['Value']
snowflakewarehouse = ssm.get_parameter(Name='/data_platform/dev/featurestore/snowflakewarehouse', WithDecryption=True)['Parameter']['Value']
snowflakeschema = ssm.get_parameter(Name='/data_platform/dev/featurestore/snowflakewareschema', WithDecryption=True)['Parameter']['Value']
postgreshost = ssm.get_parameter(Name='/data_platform/dev/featurestore/postgreshost', WithDecryption=True)['Parameter']['Value']
postgresuser = ssm.get_parameter(Name='/data_platform/dev/featurestore/postgresusername', WithDecryption=True)['Parameter']['Value'] 
snowflakerole = ssm.get_parameter(Name='/data_platform/dev/featurestore/snowflakerole', WithDecryption=True)['Parameter']['Value']  
redisconn = 'clustercfg.dev-usw2-mlops-feast.kzznmy.usw2.cache.amazonaws.com:6379,password=xc9rM^BuTGBnJXhmH<0ldejoK3Mj5weauWO,ssl=true'

In [ ]:
from feast import FeatureStore, RepoConfig
from feast.repo_config import RegistryConfig

from feast.infra.online_stores.redis import RedisOnlineStoreConfig
from feast.infra.offline_stores.snowflake import SnowflakeOfflineStoreConfig
from feast.infra.materialization.snowflake_engine import SnowflakeMaterializationEngineConfig

repo_config = RepoConfig(
    registry= RegistryConfig(
        registry_type = 'sql',
        path = f'postgresql://{postgresuser}:{postgrespwd}@{postgreshost}/{postgresdb}'
    ),
    project="feast_demo_ds",
    provider="local",
    offline_store=SnowflakeOfflineStoreConfig(
        account = snowflakeaccount,
        user = snowflakeuser,
        password = snowflakepwd,
        role = snowflakerole,
        warehouse = snowflakewarehouse,
        database = snowflakedb,
        schema = snowflakeschema
    ),
    online_store=RedisOnlineStoreConfig(
        redis_type="redis_cluster",
        connection_string = redisconn
    ),
    batch_engine=
        SnowflakeMaterializationEngineConfig(
        account = snowflakeaccount,
        user = snowflakeuser,
        password = snowflakepwd,
        role = snowflakerole,
        warehouse = snowflakewarehouse,
        database = snowflakedb,
        schema = snowflakeschema
    ),
    entity_key_serialization_version=2
)

fs = FeatureStore(config=repo_config)


In [ ]:
%%time
entity_sql = f"""
    SELECT
        NAMEORIG as USER_ID,
        TIMESTAMP as "event_timestamp"
    FROM {fs.get_data_source("transactions_source").get_table_query_string()}
    WHERE TIMESTAMP BETWEEN '2021-09-04 00:00:00+0' and '2021-09-14 00:00:00+0'
"""
training_df = fs.get_historical_features(
    entity_df=entity_sql,
    features=fs.get_feature_service("model_v2"),
).to_df()
print(training_df.head(20))

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
startdate = datetime.strptime('2021-09-10', '%Y-%m-%d')
enddate = datetime.strptime('2021-09-11', '%Y-%m-%d')

fs.materialize(start_date= startdate, end_date=enddate)

In [ ]:
features = fs.get_online_features(
    features=fs.get_feature_service("model_v1"),
    entity_rows=[
        {
            "USER_ID": "C612693043",
        }
    ],
).to_dict()

In [ ]:
def print_online_features(features):
    for key, value in sorted(features.items()):
        print(key, " : ", value)

print_online_features(features)

In [ ]:
from feast import FeatureStore, RepoConfig
from feast.repo_config import RegistryConfig

from feast.infra.online_stores.redis import RedisOnlineStoreConfig
from feast.infra.offline_stores.snowflake import SnowflakeOfflineStoreConfig
from feast.infra.materialization.snowflake_engine import SnowflakeMaterializationEngineConfig
fs4 = FeatureStore(repo_path="./feature_repo")

In [ ]:
%%time
from datetime import datetime
from dateutil.relativedelta import relativedelta
startdate = datetime.strptime('2021-01-01', '%Y-%m-%d')
enddate = datetime.strptime('2023-04-10', '%Y-%m-%d')

fs4.materialize(start_date= startdate, end_date=enddate)

In [ ]:

def get_onlinefeatures_from_featurestore(userid):
    features_online = fs4.get_online_features(
        features=fs4.get_feature_service("model_v2"),
            entity_rows=[
            {
                "USER_ID": userid,
            }
        ],
    ).to_dict()
    return features_online

In [ ]:
%%time
features = get_onlinefeatures_from_featurestore("C612693043")
def print_online_features(features):
        for key, value in sorted(features.items()):
            print(key, " : ", value)

print_online_features(features)

In [ ]:
import json
import requests

def get_online_features_from_feastserver(userid):
    online_request = {
      "feature_service": "model_v2",
      "entities": {
        "USER_ID": [userid]
      }
    }
    r = requests.post('http://feast.data-plat-dev.cloud/get-online-features', data=json.dumps(online_request))

    return r


In [ ]:
%%time
response = get_online_features_from_feastserver("C612693043")
print(json.dumps(response.json(), indent=4, sort_keys=True))